# SKRUD Prototype App
> A SKRUD client would _probably_ be implemented in static JS/CSS/HTML. This one uses a kernel per user with `voila`, and implements most of the transformations from the python side 
>> ...for now

In [ ]:
import re, traitlets as T, ipywidgets as W
from tornado import escape, httpclient, ioloop
from pyld import jsonld
from yaml import safe_dump
import random

In [ ]:
class ScrudClient(W.Textarea):
    value_parsed = T.Dict().tag(sync=True)
    context = T.Dict().tag(sync=True)
    schema = T.Dict().tag(sync=True)
    url = T.Unicode().tag(sync=True)
    headers = W.trait_types.TypedTuple(
        trait=T.Tuple(T.Unicode(), T.Unicode())
    ).tag(sync=True)
    _client = T.Instance(httpclient.AsyncHTTPClient)
    _token = T.Unicode()

    @T.default("_client")
    def _default_client(self):
        return httpclient.AsyncHTTPClient()
        
    @T.observe("url", "token")
    def _on_url(self, _):
        ioloop.IOLoop.instance().spawn_callback(lambda: self._do_fetch())
    
    def _client_headers(self):
        return {
            "Authorization": f"token {self._token}"
        }
        
    async def _do_fetch(self):
        try:
            r = await self._client.fetch(self.url, headers=self._client_headers())
        except Exception as err:
            self.value = safe_dump({"error": f"{err}"})
            return
        
        self.value_parsed = escape.json_decode(r.body)
        self.value = safe_dump(self.value_parsed)
        self.headers = list(r.headers.get_all())
        for match in [
            re.match(r'<(?P<href>.*)>.*rel="(?P<rel>[^"]*)"', link)
            for link in r.headers.get_list("Link")
        ]:
            if match is None:
                return
            groups = match.groupdict()
            if groups["rel"] == "http://www.w3.org/ns/json-ld#context":
                try:
                    self.context = escape.json_decode((await self._client.fetch(groups["href"], 
                                                                                headers=self._client_headers()
                                                                               )).body)
                except Exception as err:
                    self.context = {"errors": [f"{err}"]}
            elif groups["rel"] == "describedBy":
                try:
                    self.schema = escape.json_decode((await self._client.fetch(groups["href"], 
                                                                                headers=self._client_headers())).body)
                except Exception as err:
                    self.schema = {"errors": [f"{err}"]}

In [ ]:
url = W.Text(
    value="http://localhost:8877/cpu-temp",
    description="SCRUD URL"
)
token = W.Text(description="Token")
client = ScrudClient(description="Document", rows=16, layout=dict(width="100%"))
schema = W.Textarea(description="Schema", rows=16, layout=dict(width="100%"))
context = W.Textarea(description="Context", rows=16, layout=dict(width="100%"))
headers = W.Textarea(description="Headers", rows=16, layout=dict(width="100%"))
fetch_button = W.Button(description="Fetch")

In [ ]:
T.dlink((client, "schema"), (schema, "value"), safe_dump)
T.dlink((client, "context"), (context, "value"), safe_dump)
T.dlink((client, "headers"), (headers, "value"), safe_dump)
T.dlink((token, "value"), (client, "_token"))
T.dlink((url, "value"), (client, "url"));

In [ ]:
def _on_click(_):
    client.url = client.url.split("#")[0] + f"#{random.random()}"
fetch_button.on_click(_on_click)

In [ ]:
W.VBox([
    W.HBox([url, token, fetch_button]), 
    W.HBox([
        client,
        headers,
    ]),
    W.HBox([
        schema, 
        context
    ])
])